# recslam

> Tools for EDA of visual data collected in the RecSLAM project.


In [1]:
# default_exp tools.recslam_eda

In [2]:
#| hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
### | export
# | hide

In [4]:
from ds_contrib.tools.io.gscloud import GSBrowser
from pathlib import Path

In [14]:
## | hide

import json
import os
from pathlib import Path

from dotenv import load_dotenv

CWD = Path.cwd()
REPO_DIR = Path(*CWD.parts[: CWD.parts.index("ds_contrib") + 1])
CONFIGS_DIR = REPO_DIR / "configs"
ENV_DIR = REPO_DIR / "env/local"

with open(CONFIGS_DIR / "storage/gscloud/projects_vars.json") as f:
    projects = json.load(f)

# choose project
project = projects["dev"]
env_path = Path(ENV_DIR / f'{project["env"]}_roadly.env')

_ = load_dotenv(env_path)  # read local .env file
google_app_creds = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
roadly_cookie = os.getenv("ROADLY_COOKIE")
print(f"Initial configuration has finished:\nProject: {project}")

Initial configuration has finished:
Project: {'project': 'roadly-project-dev', 'env': 'dev', 'coldline_name': 'standard'}


In [7]:
import pandas as pd

In [8]:
def json_decoder(path: Path):
    with open(path, "r") as f:
        data = json.load(f)
    return data


def csv_decoder(path: Path):
    with open(path, "r") as f:
        data = pd.read_csv(f)
    return data

In [58]:
def get_reclslam_df(recslam_dir: Path, structure: dict):
    return pd.DataFrame.from_records(
        [(k, v["desc"], recslam_dir / v["path"]) for k, v in structure.items()],
        index="name",
        columns=["name", "desc", "path"],
    )

In [59]:
# | export
# | hide
from dataclasses import dataclass

In [60]:
# | export


@dataclass
class CameraMetadataDTO:
    field_of_view: float
    camera_type: str
    fps: int
    resolution: tuple[int, int]
    focal_length: float


@dataclass
class DeviceMetadataDTO:
    os: str
    device: str
    device_raw: str
    app_version: str
    exposure_mode: str
    max_exposure_time: float
    camera_lense_position: int
    orientation: str
    wide_camera: CameraMetadataDTO
    ultra_wide_camera: CameraMetadataDTO

    @classmethod
    def from_json(cls, json_file):
        with open(json_file) as f:
            data = json.load(f)
            wide_camera = CameraMetadataDTO(
                data["cam_fov_2"],
                data["camera_type_2"],
                data["fps2"],
                tuple(map(int, data["video_resolution_2"].split("x"))),
                data["cam_fx_2"],
            )
            ultra_wide_camera = CameraMetadataDTO(
                data["cam_fov"],
                data["camera_type"],
                data["fps"],
                tuple(map(int, data["video_resolution"].split("x"))),
                data["cam_fx"],
            )
            return cls(
                data["os"],
                data["device"],
                data["device_raw"],
                data["app_version"],
                data["exposure_mode"],
                data["max_exposure"],
                data["cam_lens_pos"],
                data["orientation"],
                wide_camera,
                ultra_wide_camera,
            )

In [61]:
# df = get_reclslam_df(SHARED_ROOT, recslam_structure)

In [62]:
# device = DeviceMetadataDTO.from_json(df.loc["device"]["path"])

In [63]:
# device.ultra_wide_camera.fps

In [64]:
# csv_decoder(df.loc["heading"]["path"])

In [65]:
# browser.cleanup()

1. Choose reclslam path
2. Download video and meta
3. Split video to frames
4. Run video browser
5. Export dataset to csv


In [66]:
sampled_datasets = pd.read_csv("/Users/arseniy/Downloads/выборка.csv")

In [67]:
sampled_datasets

,Unnamed: 0.1,Unnamed: 0,Path,Datetime,Country,City,Speed,Route,iri_mean,iri_var,...,ultrawide,video_data,name,local_path,fps,frame_count,duration,Решение,Причина,video_url
0,0,3652,Alanya/2022-07-17_09-05-31_4453D774-04F,2022-07-17 10:05:32,Turkey,NaN,13.399114,"LINESTRING (29.113155 36.661371, 29.113136 36....",10.881955,36.128544,...,True,False,2022-07-17_09-05-31_4453D774-04F,data/downloads/2022-07-17_09-05-31_4453D774-04...,5.000158,529,105.796667,Не подходит?,Брусчатка,https://storage.cloud.google.com/roadly-dev-st...
1,1,3537,2022-12-17_15-33-16_83AB9828-5B0,2022-12-17 14:33:16,Uzbekistan,NaN,13.874303,"LINESTRING (69.074585 41.123241, 69.074628 41....",11.176429,23.879309,...,True,False,2022-12-17_15-33-16_83AB9828-5B0,data/downloads/2022-12-17_15-33-16_83AB9828-5B...,5.000035,1200,239.998333,Подходит,NaN,https://storage.cloud.google.com/roadly-dev-st...
2,2,3471,2022-12-17_16-12-01_83AB9828-5B0,2022-12-17 15:12:01,Uzbekistan,Tashkent,9.621945,"LINESTRING (69.250847 41.292218, 69.250845 41....",8.533605,19.737184,...,True,False,2022-12-17_16-12-01_83AB9828-5B0,data/downloads/2022-12-17_16-12-01_83AB9828-5B...,5.000072,575,114.998333,Подходит,NaN,https://storage.cloud.google.com/roadly-dev-st...
3,3,14,2022-12-17_15-44-08_83AB9828-5B0,2022-12-17 14:44:08,Uzbekistan,NaN,13.776954,"LINESTRING (69.136316 41.178227, 69.136369 41....",10.047226,18.226671,...,True,False,2022-12-17_15-44-08_83AB9828-5B0,data/downloads/2022-12-17_15-44-08_83AB9828-5B...,5.000074,1129,225.796667,Подходит,NaN,https://storage.cloud.google.com/roadly-dev-st...
4,4,3936,2022-12-17_13-32-09_83AB9828-5B0,2022-12-17 12:32:10,Uzbekistan,NaN,10.560251,"LINESTRING (69.0679 41.129758, 69.067904 41.12...",10.391300,16.992052,...,True,False,2022-12-17_13-32-09_83AB9828-5B0,data/downloads/2022-12-17_13-32-09_83AB9828-5B...,5.000132,316,63.198333,Подходит,NaN,https://storage.cloud.google.com/roadly-dev-st...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,118,4086,Antalya/2022-03-08_12-00-17_4453,2022-03-08 13:00:16,Turkey,NaN,8.116465,"LINESTRING (32.096358 36.488346, 32.096334 36....",3.158568,0.611353,...,True,False,2022-03-08_12-00-17_4453,data/downloads/2022-03-08_12-00-17_4453.mp4,5.000138,605,120.996667,Не подходит,Мало дефектов,https://storage.cloud.google.com/roadly-dev-st...
119,119,3805,Dubai/2022-05-28_22-55-34_2A24,2022-05-28 22:55:34,United Arab Emirates,NaN,9.440000,"LINESTRING (55.37714 25.142569, 55.37721 25.14...",4.130825,0.607832,...,True,False,2022-05-28_22-55-34_2A24,data/downloads/2022-05-28_22-55-34_2A24.mp4,1.000000,7,7.000000,Не подходит,это просто стол,https://storage.cloud.google.com/roadly-dev-st...
120,120,4105,Antalya/2022-03-08_11-57-55_4453,2022-03-08 12:57:54,Turkey,NaN,6.728861,"LINESTRING (32.100665 36.485887, 32.100655 36....",3.172593,0.463560,...,True,False,2022-03-08_11-57-55_4453,data/downloads/2022-03-08_11-57-55_4453.mp4,0.000000,0,0.000000,NaN,NaN,https://storage.cloud.google.com/roadly-dev-st...
121,121,3603,Alanya/2022-06-22_21-35-56_4453D774-04F,2022-06-22 21:35:57,Georgia,Batumi,9.462348,"LINESTRING (41.61358 41.60342, 41.61361 41.603...",3.623128,0.448510,...,True,False,2022-06-22_21-35-56_4453D774-04F,data/downloads/2022-06-22_21-35-56_4453D774-04...,5.000222,375,74.996667,Не подходит,Ночь\Лужи,https://storage.cloud.google.com/roadly-dev-st...


In [68]:
items = sampled_datasets["name"].sort_values()

In [72]:
import os
import ipywidgets as widgets
from IPython.display import display


def create_items_dropdown(items):
    """Create a dropdown widget with files in the specified directory."""
    # List all files in the directory
    # Create a dropdown widget with the files
    dropdown = widgets.Dropdown(
        options=items,
        description="Files:",
        disabled=False,
    )

    # Display the widget
    display(dropdown)

    return dropdown


# Call the function with your desired directory
dropdown = create_items_dropdown(items)

Dropdown(description='Files:', options=('2021-11-21_15-17-06_4453', '2022-02-04_14-45-39_4453', '2022-02-04_14…

In [73]:
dropdown.value

'2021-11-21_15-17-06_4453'

In [74]:
chosen_video = sampled_datasets[sampled_datasets["name"] == dropdown.value][
    "video_url"
].values[0]

In [75]:
# RECSLAM_PATH = Path(
#     "roadly-dev-standard-videos/auto_6ed96cb21f4a72b8c225853250fae9ed4acdd17c82567484bffb9014d2dc2eb4"
# )
# RECSLAM_VIDEO_PATH = Path("2023-03-27_16-32-15_C305D2C3-6BF")
# LOCAL_RECSLAM_PATH = browser.downloads_path / RECSLAM_PATH / RECSLAM_VIDEO_PATH
# LOCAL_RECSLAM_PATH

In [76]:
google_app_creds = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
browser = GSBrowser(project=project["project"], credentials=google_app_creds)

In [77]:
browser.download_file(chosen_video)

INFO:ds_contrib.core.paths:Created temporary directory for downloads: `/Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmpjit4crc8`
INFO:ds_contrib.tools.io.gscloud:Downloading file from URI: `Calibration/2021-11-21_15-17-06_4453/video_2` to path: `/Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmpjit4crc8/roadly-dev-standard-videos/Calibration/2021-11-21_15-17-06_4453/video_2`
INFO:google.resumable_media._helpers:No MD5 checksum was returned from the service while downloading https://storage.googleapis.com/download/storage/v1/b/roadly-dev-standard-videos/o/Calibration%2F2021-11-21_15-17-06_4453%2Fvideo_2?generation=1685751024671958&alt=media
(which happens for composite objects), so client-side content integrity
checking is not being performed.


In [78]:
from ds_contrib.core.paths import list_paths


LOCAL_RECSLAM_PATH = list(list_paths(browser.downloads_path, recursive=True))[0].parent

In [79]:
from ds_contrib.core.data.video import sample_frames_from_video


sample_frames_from_video(
    LOCAL_RECSLAM_PATH / "video_2", LOCAL_RECSLAM_PATH / "frames", time_delta=2
)

Sampling frames:   0%|          | 0/153 [00:00<?, ?it/s]

In [80]:
# from ds_contrib.core.paths import list_paths


# list(list_paths(LOCAL_RECSLAM_PATH / "frames", recursive=True, sort=True))

In [82]:
from ds_contrib.core.paths import list_paths
from ds_contrib.tools.browser import ImageBrowser


ib = ImageBrowser(
    list(list_paths(LOCAL_RECSLAM_PATH / "frames", recursive=True, sort=True))
)
ib.browse()

In [47]:
# pd.read_csv("test.csv")

In [48]:
browser.downloads_dir.cleanup()

INFO:ds_contrib.core.paths:Cleaning up temporary directory `/Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmprx34dhgl`
